In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_csv("/kaggle/input/cinema-ticket/cinemaTicket_Ref.csv")
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

# Filling null values with mean

In [ ]:
df.isnull().sum()

In [ ]:
df['occu_perc'] = df['occu_perc'].fillna((df['occu_perc'].mean()))
df['capacity'] = df['capacity'].fillna((df['capacity'].mean()))
df.isnull().sum()

In [ ]:
df.describe()

# Correlation plot of the dataset 

**Some of the features show strong correlation with the target such as ticket sold and total sales which showed strong correlation of 0.92 in addition show-time and total sales show good correlation of 0.51 also capacity of the cinema and the total sales show correlation of 0.38.**

In [ ]:
sns.set_theme(color_codes=True)
sns.set_theme(style="darkgrid")
plt.figure(figsize = (15,5))
sns.heatmap(df.corr()  ,annot = True , cmap="cividis")

# Investigating features that have high correlation with the target 

In [ ]:
%matplotlib inline
sns.set_theme(color_codes=True)
sns.set_theme(style="darkgrid")
plt.figure(figsize = (15,5))
plt.xlabel('total_sales')
plt.ylabel('ticket_use')
plt.scatter(df.total_sales,df.ticket_use,color='blue',marker='+')

In [ ]:
%matplotlib inline
sns.set_theme(color_codes=True)
sns.set_theme(style="darkgrid")
plt.figure(figsize = (15,5))
plt.xlabel('total_sales')
plt.ylabel('capacity')
plt.scatter(df.total_sales,df.capacity,color='blue',marker='+')

In [ ]:
%matplotlib inline
sns.set_theme(color_codes=True)
sns.set_theme(style="darkgrid")
plt.figure(figsize = (15,5))
plt.xlabel('total_sales')
plt.ylabel('tickets_sold')
plt.scatter(df.total_sales,df.tickets_sold,color='blue',marker='+')

In [ ]:
%matplotlib inline
sns.set_theme(color_codes=True)
sns.set_theme(style="darkgrid")
plt.figure(figsize = (15,5))
plt.xlabel('total_sales')
plt.ylabel('show_time')
plt.scatter(df.total_sales,df.show_time,color='blue',marker='+')

In [ ]:
%matplotlib inline
sns.set_theme(color_codes=True)
sns.set_theme(style="darkgrid")
plt.figure(figsize = (15,5))
plt.xlabel('total_sales')
plt.ylabel('ticket_price')
plt.scatter(df.total_sales,df.ticket_price,color='blue',marker='+')

# Target normalization using log scale

In [ ]:
f= plt.figure(figsize=(15,5))

ax=f.add_subplot(121)
sns.distplot(df['total_sales'],bins=50,color='r',ax=ax)
ax.set_title('Distribution of total sales')

ax=f.add_subplot(122)
sns.distplot(np.log10(df['total_sales']),bins=40,color='b',ax=ax)
ax.set_title('Distribution of total sales in $log$ sacle')
ax.set_xscale('log');

# Spliting the data into training and testing

In [ ]:
from sklearn.model_selection import train_test_split
X = df[['tickets_sold','occu_perc',
             'capacity', 'show_time','ticket_price','ticket_use']]
y = np.log10(df['total_sales'])
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.30)

# Data normalization 

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
x_train = scaling.transform(X_train)
x_test = scaling.transform(X_test)

# Linear regression 

**only features which has high correlation with the target is selected for the model building**

In [ ]:
from sklearn import linear_model
# Create linear regression object
reg = linear_model.LinearRegression()
reg.fit(x_train,y_train)

In [ ]:
y_pred = reg.predict(x_test)

In [ ]:
sns.set_theme(color_codes=True)
sns.set_theme(style="darkgrid")
plt.figure(figsize = (15,5))
ax1 = sns.distplot(y_test, hist=False, color='r', label='Actual Value')
sns.distplot(y_pred, hist=False, color='b', label='Fitted Values', ax=ax1)
ax1.ticklabel_format(style='plain')
plt.legend(loc='best')
plt.show()

# Model evaluation

In [ ]:
import sklearn.metrics as metrics
mae = metrics.mean_absolute_error(y_test, y_pred)
mse = metrics.mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse) # or mse**(0.5)  
r2 = metrics.r2_score(y_test,y_pred)

print("Results of sklearn.metrics:")
print("MAE:",mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R-Squared:", r2)

#### the predicted values and the actual values are qiute close 

# Non linear regression (knn)
**let's see if we can improve the performance of the model using non linear regression (knn)**

In [ ]:
df_actual_pre ={'Actual': y_test, 'Predicted': y_pred}
df1_actual_pre= pd.DataFrame(df_actual_pre)
df1_actual_pre.head(10)

In [ ]:
from sklearn import neighbors
n_neighbors = 3

for i, weights in enumerate(["uniform", "distance"]):
    knn = neighbors.KNeighborsRegressor(n_neighbors, weights=weights, leaf_size=3)
    knn.fit(x_train, y_train)
    y_pred2 = knn.predict(x_test)

In [ ]:
knn

# model evaluation 

In [ ]:
import sklearn.metrics as metrics
import numpy as np
mae = metrics.mean_absolute_error(y_test, y_pred2)
mse = metrics.mean_squared_error(y_test, y_pred2)
rmse = np.sqrt(mse) # or mse**(0.5)
r2 = metrics.r2_score(y_test,y_pred2)
print("Results of sklearn.metrics:")
print("MAE:",mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R-Squared:", r2)

In [ ]:
df_actual_pre2 ={'Actual': y_test, 'Predicted': y_pred2}
df1_actual_pre2= pd.DataFrame(df_actual_pre2)
df1_actual_pre2.head(10)

# conclusion

**When comparing multiple linear regression and nonlinear regression (the kNN regressor), the kNN regressor outperformed the linear one across all matrices. The KNN regressor obtained very low MAE, RMSE, and MSE, and a high R-squared compared to that of multiple linear regression. Even though the data showed some sort of linearity, but still the linear model did not compete with the nonlinear KNN regressor model on this dataset.**
